In [3]:
!pip install snowflake-snowpark-python


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip


In [4]:
import os
from pprint import pprint
import re
import datetime

In [5]:
from snowflake.snowpark import (
    Column,
    DataFrame,
    Session,
    Window
)
from snowflake.snowpark import functions as f
from snowflake.snowpark.types import IntegerType, StringType, StructType, DateType, StructField, MapType

In [8]:
connection_parameters = {
"account": "gi02106.eu-west-2.aws",
"user": "pujaverma",
"password": "Itzme#123",
"role": "accountadmin",
"warehouse": "workshopwh",
"database": "workshopdb",
"schema": "workshopsch",
}

In [9]:
snowflake_conn_session = Session.builder.configs(connection_parameters).create()

In [10]:
snowflake_conn_session.sql("select current_warehouse(), current_database(), current_schema()").show()

---------------------------------------------------------------------
|"CURRENT_WAREHOUSE()"  |"CURRENT_DATABASE()"  |"CURRENT_SCHEMA()"  |
---------------------------------------------------------------------
|WORKSHOPWH             |WORKSHOPDB            |WORKSHOPSCH         |
---------------------------------------------------------------------



In [11]:
df = snowflake_conn_session.table("Snowpark_dataset")

In [12]:
df.schema.fields

[StructField('ID', LongType(), nullable=True),
 StructField('CHECKING_STATUS', StringType(), nullable=True),
 StructField('DURATION', LongType(), nullable=True),
 StructField('CREDIT_HISTORY', StringType(), nullable=True),
 StructField('PURPOSE', StringType(), nullable=True),
 StructField('CREDIT_AMOUNT', LongType(), nullable=True),
 StructField('SAVINGS_STATUS', StringType(), nullable=True),
 StructField('EMPLOYMENT', StringType(), nullable=True),
 StructField('INSTALLMENT_COMMITMENT', LongType(), nullable=True),
 StructField('PERSONAL_STATUS', StringType(), nullable=True),
 StructField('THER_PARTIES', StringType(), nullable=True),
 StructField('RESIDENCE_SINCE', LongType(), nullable=True),
 StructField('PROPERTY_MAGNITUDE', StringType(), nullable=True),
 StructField('AGE', StringType(), nullable=True),
 StructField('OTHER_PAYMENT_PLANS', StringType(), nullable=True),
 StructField('HOUSING', StringType(), nullable=True),
 StructField('EXISTING_CREDITS', LongType(), nullable=True),
 St

In [13]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.sklearn.processing import SKLearnProcessor

region = boto3.session.Session().region_name

role = get_execution_role()
sklearn_processor = SKLearnProcessor(
    framework_version="0.20.0", role=role, instance_type="ml.m5.xlarge", instance_count=1
)

/opt/conda/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [18]:
import pandas as pd

input_data = "s3://sagemaker-workshopdata/census-income.csv"
df = pd.read_csv(input_data, nrows=10)
df.printSchema
#df.head(n=10)

AttributeError: 'DataFrame' object has no attribute 'printSchema'